In [1]:
import mysql.connector
db_config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'bd2024',
    'port': 3306
}

In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import mysql.connector
import cx_Oracle
from tkinter import Menu
import pymysql

In [8]:
def lista_mysql():
    mysql_conn = pymysql.connect(
            user=db_config["user"],
            password=db_config["password"],
            host=db_config["host"],
            port=int(db_config["port"]),
            database=banco_selecionado
        )
    cursor = mysql_conn.cursor()
    cursor.execute("SHOW TABLES;")
    tabelas_existentes_mysql = [linha[0].lower() for linha in mysql_cursor.fetchall()]
    return tabelas_existentes_mysql

In [2]:
def lista_oracle(user_, password_):
            dsn = "localhost:1521/XEPDB1"
            conn = cx_Oracle.connect(user=user_,password=password_,dsn=dsn)
            cursor_oracle = conn.cursor()   
        
            try:
                cursor_oracle.execute("SELECT table_name FROM user_tables")
                tabelas_existentes_oracle = [linha[0] for linha in cursor_oracle.fetchall()]
            except cx_Oracle.DatabaseError as e:
                print(f"Erro ao executar a consulta no Oracle: {e}")
            finally:
                cursor_oracle.close()
                conn.close()
            return tabelas_existentes_oracle

In [ ]:
def todas_tabelas (direcao):
    if direcao == "mysql_to_oracle":
        
    elif direcao == 'oracle_to_mysql':

In [ ]:
db_config = {}
db_configorcl = {}

def filtraTabelasExistentes(lista, lista2):
    lista = [item.upper() for item in lista]
    list_filtrada = []
    for item in lista2:
        if item.upper() in lista:
            print("Já existe a tabela:", item)
        else:
            list_filtrada.append(item)
            print("Não existe a tabela:", item)
    return list_filtrada

def verifica_relacionamento(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura, direcao):
    if direcao == "mysql_to_oracle":
        mysql_conn = pymysql.connect(
            user=db_config["user"],
            password=db_config["password"],
            host=db_config["host"],
            port=int(db_config["port"]),
            database=banco_selecionado
        )
        cursor = mysql_conn.cursor()
        cursor.callproc("sp_tabela_necessaria", (banco_selecionado, tabela_selecionada))
        resultados = cursor.fetchall()
        if resultados:
            for result in resultados:
                migrar_banco_mysql_para_oracle(banco_selecionado, result[0], migrar_estrutura_dados, migrar_somente_estrutura)
            migrar_banco_mysql_para_oracle(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura)
        else:
            migrar_banco_mysql_para_oracle(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura)
    elif direcao == 'oracle_to_mysql':
        dsn = cx_Oracle.makedsn(db_configorcl["host"], db_configorcl["port"], service_name=db_configorcl["service_name"])
        conn = cx_Oracle.connect(user=db_configorcl["user"], password=db_configorcl["password"], dsn=dsn)
        cursor_oracle = conn.cursor()
        
        query = f"""
            SELECT
                 b.table_name AS referenced_table
            FROM
                all_cons_columns a
                JOIN all_constraints c ON a.constraint_name = c.constraint_name
                JOIN all_cons_columns b ON c.r_constraint_name = b.constraint_name
            WHERE
                a.table_name = '{tabela_selecionada.upper()}'
                AND a.owner = '{db_configorcl["user"].upper()}'
                AND c.constraint_type = 'R'
        """
        
        try:
            cursor_oracle.execute(query)
            resultados_oracle = cursor_oracle.fetchall()

            if resultados_oracle:
                for result in resultados_oracle:
                    migrar_banco_oracle_para_mysql(banco_selecionado, result[0], migrar_estrutura_dados, migrar_somente_estrutura)
                migrar_banco_oracle_para_mysql(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura)
            else:
                migrar_banco_oracle_para_mysql(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura)

        except cx_Oracle.DatabaseError as e:
            print(f"Erro ao executar a consulta no Oracle: {e}")
        finally:
            cursor_oracle.close()
            conn.close()
   

def migrar_banco_mysql_para_oracle(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura):
    mysql_conn = None
    try:
        dsn = cx_Oracle.makedsn(db_configorcl["host"], db_configorcl["port"], service_name=db_configorcl["service_name"])
        conn = cx_Oracle.connect(user=db_configorcl["user"], password=db_configorcl["password"], dsn=dsn)

        print("Conexão com o banco de dados Oracle bem-sucedida!")

        mysql_conn = mysql.connector.connect(
            user=db_config["user"],
            password=db_config["password"],
            host=db_config["host"],
            port=int(db_config["port"]),
            database=banco_selecionado
        )

        mysql_connection = pymysql.connect(
            user=db_config["user"],
            password=db_config["password"],
            host=db_config["host"],
            port=int(db_config["port"]),
            database=banco_selecionado
        )
        
        
        mysql_cursor = mysql_conn.cursor()
        mysql_cursor.execute(f"DESCRIBE {tabela_selecionada};")
        columns = mysql_cursor.fetchall()
        
        cursor_oracle = conn.cursor()
        cursor_oracle.execute("SELECT table_name FROM user_tables")
        tabelas_existentes_oracle = [linha[0] for linha in cursor_oracle.fetchall()]

        if tabela_selecionada.upper() in tabelas_existentes_oracle:
            messagebox.showinfo("Tabela Já Existe", f"A tabela '{tabela_selecionada}' já existe no banco de dados Oracle e não será migrada.")
            return

        # Identificar a chave primária
        mysql_cursor.execute(f"SHOW KEYS FROM {tabela_selecionada} WHERE Key_name = 'PRIMARY';")
        primary_keys = mysql_cursor.fetchall()
        pk_columns = [pk[4] for pk in primary_keys]

        # Identificar as chaves estrangeiras
        procedure_nome = 'sp_identifica_chaves_estrangeiras'
        parametros = (tabela_selecionada, banco_selecionado)  # Coloque os parâmetros aqui, se necessário

        pymysql_cursor = mysql_connection.cursor()
        pymysql_cursor.callproc(procedure_nome, parametros)
        foreign_keys = pymysql_cursor.fetchall()
        print(foreign_keys)

        # Criar a tabela
        create_table_sql = f'CREATE TABLE "{tabela_selecionada.upper()}" ('

        for column in columns:
            column_name = column[0]
            column_type = column[1]

            if "int" in column_type.lower():
                oracle_type = "NUMBER"
            elif "varchar" in column_type.lower():
                length = column_type.split('(')[1].split(')')[0]
                oracle_type = f"VARCHAR2({length})"
            elif "char" in column_type.lower():
                length = column_type.split('(')[1].split(')')[0]
                oracle_type = f"CHAR({length})"
            elif "text" in column_type.lower():
                oracle_type = "CLOB"
            elif "datetime" in column_type.lower():
                oracle_type = "DATE"
            else:
                oracle_type = "VARCHAR2(255)"

            create_table_sql += f'"{column_name.upper()}" {oracle_type}, '

        if pk_columns:
            pk_columns_str = ", ".join([f'"{pk.upper()}"' for pk in pk_columns])
            create_table_sql += f"PRIMARY KEY ({pk_columns_str}), "

        create_table_sql = create_table_sql.rstrip(", ") + ")"
        
        print(f"Executing: {create_table_sql}")
        cursor_oracle.execute(create_table_sql)

        # Adicionar chaves estrangeiras
        added_constraints = set()
        for fk in foreign_keys:
            column_name, ref_table, ref_column = fk
            constraint_name = f"fk_{column_name.lower()}"
            if constraint_name not in added_constraints:
                if ref_table.upper() in tabelas_existentes_oracle:
                    alter_sql = f"""
                        ALTER TABLE "{tabela_selecionada.upper()}"
                        ADD CONSTRAINT {constraint_name}
                        FOREIGN KEY ("{column_name.upper()}")
                        REFERENCES "{ref_table.upper()}" ("{ref_column.upper()}")
                    """
                    print(f"Executing: {alter_sql}")
                    cursor_oracle.execute(alter_sql)
                    added_constraints.add(constraint_name)

        # Migração dos dados
        if migrar_estrutura_dados:
            mysql_cursor.execute(f"SELECT * FROM {tabela_selecionada};")
            rows = mysql_cursor.fetchall()
            for row in rows:
                placeholders = ", ".join([":" + str(i + 1) for i in range(len(row))])
                insert_sql = f"INSERT INTO \"{tabela_selecionada.upper()}\" VALUES ({placeholders})"
                cursor_oracle.execute(insert_sql, row)

        conn.commit()

        messagebox.showinfo("Migração concluída", "Migração do banco de dados concluída com sucesso!")

    except cx_Oracle.Error as error:
        messagebox.showerror("Erro de Migração", f"Erro ao migrar o banco de dados: {error}")

    finally:
        if mysql_conn is not None:
            mysql_conn.close()
        conn.close()


def migrar_banco_oracle_para_mysql(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura):
    oracle_conn = None
    try:
        dsn = cx_Oracle.makedsn(db_configorcl["host"], db_configorcl["port"], service_name=db_configorcl["service_name"])
        oracle_conn = cx_Oracle.connect(user=db_configorcl["user"], password=db_configorcl["password"], dsn=dsn)

        print("Conexão com o banco de dados Oracle bem-sucedida!")

        mysql_conn = mysql.connector.connect(
            user=db_config["user"],
            password=db_config["password"],
            host=db_config["host"],
            port=int(db_config["port"]),
            database=banco_selecionado
        )
        
        mysql_cursor = mysql_conn.cursor()
        cursor_oracle = oracle_conn.cursor()

        cursor_oracle.execute(f"""
            SELECT column_name, data_type, data_length, data_precision, data_scale
            FROM all_tab_columns
            WHERE table_name = '{tabela_selecionada.upper()}' and owner = '{db_configorcl["user"]}'
        """)
        columns = cursor_oracle.fetchall()
        
        mysql_cursor.execute("SHOW TABLES;")
        tabelas_existentes_mysql = [linha[0].lower() for linha in mysql_cursor.fetchall()]

        if tabela_selecionada.lower() in tabelas_existentes_mysql:
            messagebox.showinfo("Tabela Já Existe", f"A tabela '{tabela_selecionada}' já existe no banco de dados MySQL e não será migrada.")
            return

        # Inicializa a string de criação da tabela
        create_table_sql = f'CREATE TABLE `{tabela_selecionada}` ('
        colunas_adicionadas = set()
        pk_columns = []

        # Consultar as chaves primárias
        cursor_oracle.execute(f"""
            SELECT cols.column_name
            FROM all_constraints cons
            JOIN all_cons_columns cols ON cons.constraint_name = cols.constraint_name
            WHERE cons.constraint_type = 'P'
            AND cols.table_name = '{tabela_selecionada.upper()}'
            AND cons.owner = '{db_configorcl["user"]}'
        """)
        pk_columns = [pk[0] for pk in cursor_oracle.fetchall()]

        # Consultar as chaves estrangeiras
        cursor_oracle.execute(f"""
            SELECT a.column_name, c.table_name AS ref_table, c.column_name AS ref_column
            FROM all_cons_columns a
            JOIN all_constraints b ON a.constraint_name = b.constraint_name
            JOIN all_cons_columns c ON b.r_constraint_name = c.constraint_name
            WHERE b.constraint_type = 'R'
            AND a.table_name = '{tabela_selecionada.upper()}'
            AND a.owner = '{db_configorcl["user"]}'
        """)
        foreign_keys = cursor_oracle.fetchall()

        for column in columns:
            column_name = column[0]
            column_type = column[1]
            column_length = column[2]
            precision = column[3]
            scale = column[4]
        
            if "NUMBER" in column_type.upper() and precision is not None:
                if scale is not None and scale > 0:
                    mysql_type = f"DECIMAL({precision},{scale})"
                else:
                    mysql_type = f"DECIMAL({precision})"
            elif "NUMBER" in column_type.upper():
                mysql_type = "INT"
            elif "VARCHAR2" in column_type.upper():
                mysql_type = f"VARCHAR({column_length})"
            elif "CHAR" in column_type.upper():
                mysql_type = f"CHAR({column_length})"
            elif "CLOB" in column_type.upper():
                mysql_type = "TEXT"
            elif "DATE" in column_type.upper():
                mysql_type = "DATETIME"
            else:
                mysql_type = f"VARCHAR({column_length})"
        
            create_table_sql += f'`{column_name.lower()}` {mysql_type}, '

        if pk_columns:
            pk_columns_str = ", ".join([f"`{pk.lower()}`" for pk in pk_columns])
            create_table_sql += f"PRIMARY KEY ({pk_columns_str}), "
    
        create_table_sql = create_table_sql.rstrip(", ") + ")"
        
        print(f"Executing: {create_table_sql}")
        mysql_cursor.execute(create_table_sql)
    
        # Adicionar chaves estrangeiras
        for fk in foreign_keys:
            column_name, ref_table, ref_column = fk
            if ref_table.lower() in tabelas_existentes_mysql:
                alter_sql = f"""
                    ALTER TABLE `{tabela_selecionada}`
                    ADD CONSTRAINT fk_{column_name.lower()}
                    FOREIGN KEY (`{column_name.lower()}`)
                    REFERENCES `{ref_table.lower()}` (`{ref_column}`)
                """
                print(f"Executing: {alter_sql}")
                mysql_cursor.execute(alter_sql)
    
        # Migração dos dados
        if migrar_estrutura_dados:
            cursor_oracle.execute(f"SELECT * FROM {tabela_selecionada.upper()}")
            rows = cursor_oracle.fetchall()
            for row in rows:
                placeholders = ", ".join(["%s" for _ in row])
                insert_sql = f"INSERT INTO `{tabela_selecionada}` VALUES ({placeholders})"
                mysql_cursor.execute(insert_sql, row)
        
        mysql_conn.commit()
        messagebox.showinfo("Migração concluída", "Migração do banco de dados concluída com sucesso!")
        
    except (cx_Oracle.Error, mysql.connector.Error) as error:
        messagebox.showerror("Erro de Migração", f"Erro ao migrar o banco de dados: {error}")
    
    finally:
        if oracle_conn is not None:
            oracle_conn.close()
        if mysql_conn is not None:
            mysql_conn.close()

def janela_tabela(banco_selecionado, direcao):
    janela4 = tk.Toplevel()
    janela4.title("CFSW MIGRAÇÃO DE DADOS")
    janela4.geometry("900x450")  
    janela4.resizable(False, False)

    try:
        if direcao == "mysql_to_oracle":
            cnx = mysql.connector.connect(
                user=db_config["user"],
                password=db_config["password"],
                host=db_config["host"],
                port=int(db_config["port"]),
                database=banco_selecionado
            )
            cursor = cnx.cursor()
            cursor.execute("SHOW TABLES")
            tabelas_mysql = [tabela[0] for tabela in cursor.fetchall()]
            cnx.close()
            
            dsn = cx_Oracle.makedsn(db_configorcl["host"], db_configorcl["port"], service_name=db_configorcl["service_name"])
            conn = cx_Oracle.connect(user=db_configorcl["user"], password=db_configorcl["password"], dsn=dsn)
            cursor = conn.cursor()
            cursor.execute("SELECT table_name FROM user_tables")
            tabelas_oracle = [linha[0] for linha in cursor.fetchall()]

            tabelas = filtraTabelasExistentes(tabelas_oracle, tabelas_mysql)

        
        else:

            cnx = mysql.connector.connect(
                user=db_config["user"],
                password=db_config["password"],
                host=db_config["host"],
                port=int(db_config["port"]),
                database=banco_selecionado
            )
            cursor = cnx.cursor()
            cursor.execute("SHOW TABLES")
            tabelas_mysql = [tabela[0] for tabela in cursor.fetchall()]
            cnx.close()
            
            dsn = cx_Oracle.makedsn(db_configorcl["host"], db_configorcl["port"], service_name=db_configorcl["service_name"])
            conn = cx_Oracle.connect(user=db_configorcl["user"], password=db_configorcl["password"], dsn=dsn)
            cursor = conn.cursor()
            cursor.execute("SELECT table_name FROM user_tables")
            tabelas_oracle = [linha[0] for linha in cursor.fetchall()]
            conn.close()

            tabelas = filtraTabelasExistentes(tabelas_mysql, tabelas_oracle)

        if not tabelas:
            messagebox.showinfo("Sem Tabelas", "Não há tabelas disponíveis para migração.")
            janela4.destroy()
            return
        
        scrollbar = tk.Scrollbar(janela4)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        lista_tabelas = tk.Listbox(janela4, yscrollcommand=scrollbar.set, selectmode=tk.SINGLE)
        lista_tabelas.pack(pady=20, padx=50)

        scrollbar.config(command=lista_tabelas.yview)

        for tabela in tabelas:
            lista_tabelas.insert(tk.END, tabela)

        def migrar_tabela():
            tabela_selecionada = lista_tabelas.get(tk.ACTIVE)
            if not tabela_selecionada:
                messagebox.showwarning("Nenhuma Tabela Selecionada", "Por favor, selecione uma tabela para migrar.")
                return

            migrar_estrutura_dados = messagebox.askyesno("Migrar Estrutura e Dados", "Você deseja migrar a estrutura e os dados da tabela?")
            migrar_somente_estrutura = False

            if not migrar_estrutura_dados:
                migrar_somente_estrutura = messagebox.askyesno("Migrar Apenas Estrutura", "Você deseja migrar apenas a estrutura da tabela?")

            verifica_relacionamento(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura, direcao)

        botao_migrar = tk.Button(janela4, text="Migrar Tabela Selecionada", command=migrar_tabela)
        botao_migrar.pack(pady=20)

    except (mysql.connector.Error, cx_Oracle.Error) as error:
        messagebox.showerror("Erro ao Conectar", f"Erro ao conectar com o banco de dados: {error}")
        janela4.destroy()

def conectar():
    banco_selecionado = banco_entry.get()
    direcao = direcao_var.get()
    if not banco_selecionado:
        messagebox.showwarning("Banco Não Selecionado", "Por favor, selecione um banco de dados.")
        return

    janela_tabela(banco_selecionado, direcao)

def sair():
    janela.destroy()

def janela_principal():
    global janela, banco_entry, direcao_var
    janela = tk.Tk()
    janela.title("CFSW MIGRAÇÃO DE DADOS")
    janela.geometry("900x500")
    janela.resizable(False, False)

    # Criação do menu
    barra_menu = Menu(janela)
    menu_arquivo = Menu(barra_menu, tearoff=0)
    menu_arquivo.add_command(label="Sair", command=sair)
    barra_menu.add_cascade(label="Arquivo", menu=menu_arquivo)
    janela.config(menu=barra_menu)

    direcao_var = tk.StringVar()
    direcao_var.set("mysql_to_oracle")

    # Frame para opções de direção
    direcao_frame = tk.Frame(janela)
    direcao_frame.pack(pady=20)

    mysql_to_oracle_radio = tk.Radiobutton(direcao_frame, text="MySQL para Oracle", variable=direcao_var, value="mysql_to_oracle")
    oracle_to_mysql_radio = tk.Radiobutton(direcao_frame, text="Oracle para MySQL", variable=direcao_var, value="oracle_to_mysql")

    mysql_to_oracle_radio.grid(row=0, column=0, padx=20)
    oracle_to_mysql_radio.grid(row=0, column=1, padx=20)

    banco_label = tk.Label(janela, text="Escolha o banco de dados:")
    banco_label.pack()

    banco_entry = ttk.Combobox(janela, width=40)
    banco_entry.pack()

    def carregar_bancos_mysql():
        global db_config
        try:
            mysql_conn = pymysql.connect(
                user=db_config["user"],
                password=db_config["password"],
                host=db_config["host"],
                port=int(db_config["port"]),
            )
            cursor = mysql_conn.cursor()
            cursor.execute("SHOW DATABASES")
            databases = [database[0] for database in cursor.fetchall()]
            banco_entry["values"] = databases
            cursor.close()
            mysql_conn.close()
        except pymysql.Error as error:
            messagebox.showerror("Erro de Conexão", f"Erro ao carregar bancos de dados: {error}")

    def carregar_bancos_oracle():
        global db_configorcl
        try:
            dsn = cx_Oracle.makedsn(db_configorcl["host"], db_configorcl["port"], service_name=db_configorcl["service_name"])
            conn = cx_Oracle.connect(user=db_configorcl["user"], password=db_configorcl["password"], dsn=dsn)
            cursor = conn.cursor()
            cursor.execute("SELECT username FROM all_users")
            users = [user[0] for user in cursor.fetchall()]
            banco_entry["values"] = users
            cursor.close()
            conn.close()
        except cx_Oracle.Error as error:
            messagebox.showerror("Erro de Conexão", f"Erro ao carregar bancos de dados: {error}")

    def definir_banco():
        direcao = direcao_var.get()
        if direcao == "mysql_to_oracle":
            carregar_bancos_mysql()
        else:
            carregar_bancos_oracle()

    definir_banco()

    conectar_btn = tk.Button(janela, text="Conectar", command=conectar)
    conectar_btn.pack(pady=20)

    janela.mainloop()

def login_bancos():
    def conectar_bancos():
        global db_config, db_configorcl
        db_config = {
            "user": mysql_user_entry.get(),
            "password": mysql_pass_entry.get(),
            "host": mysql_host_entry.get(),
            "port": mysql_port_entry.get()
        }
        db_configorcl = {
            "user": oracle_user_entry.get(),
            "password": oracle_pass_entry.get(),
            "host": oracle_host_entry.get(),
            "port": oracle_port_entry.get(),
            "service_name": oracle_serv_entry.get()
        }
        login_window.destroy()
        janela_principal()

    login_window = tk.Tk()
    login_window.title("CFSW MIGRAÇÃO DE DADOS")
    login_window.geometry("600x600")
    login_window.resizable(False, False)

    tk.Label(login_window, text="MySQL Login", font=('Helvetica', 12, 'bold')).pack(pady=10)
    mysql_user_label = tk.Label(login_window, text="Usuário:")
    mysql_user_label.pack()
    mysql_user_entry = tk.Entry(login_window)
    mysql_user_entry.pack()

    mysql_pass_label = tk.Label(login_window, text="Senha:")
    mysql_pass_label.pack()
    mysql_pass_entry = tk.Entry(login_window, show="*")
    mysql_pass_entry.pack()

    mysql_host_label = tk.Label(login_window, text="Host:")
    mysql_host_label.pack()
    mysql_host_entry = tk.Entry(login_window)
    mysql_host_entry.pack()

    mysql_port_label = tk.Label(login_window, text="Porta:")
    mysql_port_label.pack()
    mysql_port_entry = tk.Entry(login_window)
    mysql_port_entry.pack()

    tk.Label(login_window, text="Oracle Login", font=('Helvetica', 12, 'bold')).pack(pady=10)
    oracle_user_label = tk.Label(login_window, text="Usuário:")
    oracle_user_label.pack()
    oracle_user_entry = tk.Entry(login_window)
    oracle_user_entry.pack()

    oracle_pass_label = tk.Label(login_window, text="Senha:")
    oracle_pass_label.pack()
    oracle_pass_entry = tk.Entry(login_window, show="*")
    oracle_pass_entry.pack()

    oracle_host_label = tk.Label(login_window, text="Host:")
    oracle_host_label.pack()
    oracle_host_entry = tk.Entry(login_window)
    oracle_host_entry.pack()

    oracle_port_label = tk.Label(login_window, text="Porta:")
    oracle_port_label.pack()
    oracle_port_entry = tk.Entry(login_window)
    oracle_port_entry.pack()

    oracle_serv_label = tk.Label(login_window, text="Service Name:")
    oracle_serv_label.pack()
    oracle_serv_entry = tk.Entry(login_window)
    oracle_serv_entry.pack()

    connect_btn = tk.Button(login_window, text="Conectar", command=conectar_bancos)
    connect_btn.pack(pady=20)

    login_window.mainloop()

login_bancos()

Não existe a tabela: auth_group
Não existe a tabela: auth_group_permissions
Não existe a tabela: auth_permission
Não existe a tabela: auth_user
Não existe a tabela: auth_user_groups
Não existe a tabela: auth_user_user_permissions
Não existe a tabela: authtoken_token
Não existe a tabela: avaliacao
Não existe a tabela: carrinho
Não existe a tabela: categoria_produto
Não existe a tabela: django_admin_log
Não existe a tabela: django_content_type
Não existe a tabela: django_migrations
Não existe a tabela: django_session
Não existe a tabela: formapagamento
Não existe a tabela: ong
Não existe a tabela: pessoa
Não existe a tabela: pet
Não existe a tabela: pet_adocao
Não existe a tabela: pet_foto
Não existe a tabela: pet_porte
Não existe a tabela: pet_raca
Não existe a tabela: pet_tipo
Não existe a tabela: petshop
Não existe a tabela: produto
Não existe a tabela: produto_foto
Não existe a tabela: servico
Não existe a tabela: solicita
Não existe a tabela: tentativa_adota
Não existe a tabela: tip

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\visao_am\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_1216\104618667.py", line 403, in migrar_tabela
    verifica_relacionamento(banco_selecionado, tabela_selecionada, migrar_estrutura_dados, migrar_somente_estrutura, direcao)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_1216\104618667.py", line 25, in verifica_relacionamento
    cursor.callproc("sp_tabela_necessaria", (banco_selecionado, tabela_selecionada))
  File "C:\Users\user\anaconda3\envs\visao_am\lib\site-packages\pymysql\cursors.py", line 269, in callproc
    self._query(q)
  File "C:\Users\user\anaconda3\envs\visao_am\lib\site-packages\pymysql\cursors.py", line 322, in _query
    conn.query(q)
  File "C:\Users\user\anaconda3\envs\visao_am\lib\site-packages\pymysql\connections.py", line 563, in query
    self._affected_rows = self._read_qu

Não existe a tabela: altsalario
Não existe a tabela: bairro
Não existe a tabela: cidade
Não existe a tabela: cliente
Não existe a tabela: clientestatus
Não existe a tabela: estadocivil
Não existe a tabela: filial
Não existe a tabela: formapagamento
Não existe a tabela: fornecedor
Não existe a tabela: fornecedorfone
Não existe a tabela: funcionario
Não existe a tabela: grupoproduto
Não existe a tabela: itemvenda
Não existe a tabela: juros
Não existe a tabela: produto
Não existe a tabela: transacao
Não existe a tabela: venda
Não existe a tabela: vendedor
Não existe a tabela: zona
Conexão com o banco de dados Oracle bem-sucedida!
()
Executing: CREATE TABLE "CIDADE" ("CIDCODIGO" NUMBER, "CIDNOME" VARCHAR2(80), PRIMARY KEY ("CIDCODIGO"))


In [5]:
lista_oracle("CADU","root")

['CIDADE']

In [1]:
import tkinter as tk
from tkinter import messagebox

def verificar_selecao():
    if not var_estrutura_dados.get() and not var_apenas_estrutura.get():
        messagebox.showwarning("Seleção Inválida", "Você deve selecionar 'Estrutura e Dados' ou 'Apenas Estrutura'.")
        return False
    return True

def selecionar_estrutura_dados():
    if var_estrutura_dados.get():
        var_apenas_estrutura.set(False)

def selecionar_apenas_estrutura():
    if var_apenas_estrutura.get():
        var_estrutura_dados.set(False)

def mostrar_opcoes():
    if not verificar_selecao():
        return

    todas_tabelas = var_todas_tabelas.get()
    estrutura_dados = var_estrutura_dados.get()
    apenas_estrutura = var_apenas_estrutura.get()

    opcoes_selecionadas = []
    if todas_tabelas:
        opcoes_selecionadas.append("Todas as Tabelas")
    if estrutura_dados:
        opcoes_selecionadas.append("Estrutura e Dados")
    if apenas_estrutura:
        opcoes_selecionadas.append("Apenas Estrutura")

    messagebox.showinfo("Opções Selecionadas", f"Você selecionou: {', '.join(opcoes_selecionadas)}")

def fechar_janela():
    if verificar_selecao():
        janela.destroy()

# Criação da janela principal
janela = tk.Tk()
janela.title("Opções de Migração")
janela.geometry("300x200")

# Variáveis para armazenar o estado das checkboxes
var_todas_tabelas = tk.BooleanVar()
var_estrutura_dados = tk.BooleanVar()
var_apenas_estrutura = tk.BooleanVar()

# Criação das checkboxes
checkbox_todas_tabelas = tk.Checkbutton(janela, text="Todas as Tabelas", variable=var_todas_tabelas)
checkbox_estrutura_dados = tk.Checkbutton(janela, text="Estrutura e Dados", variable=var_estrutura_dados, command=selecionar_estrutura_dados)
checkbox_apenas_estrutura = tk.Checkbutton(janela, text="Apenas Estrutura", variable=var_apenas_estrutura, command=selecionar_apenas_estrutura)

# Posicionamento das checkboxes na janela
checkbox_todas_tabelas.pack(pady=5)
checkbox_estrutura_dados.pack(pady=5)
checkbox_apenas_estrutura.pack(pady=5)

# Botão para mostrar as opções selecionadas
botao_mostrar = tk.Button(janela, text="Mostrar Opções Selecionadas", command=mostrar_opcoes)
botao_mostrar.pack(pady=10)

# Verificação para garantir que uma das opções seja selecionada ao fechar a janela
janela.protocol("WM_DELETE_WINDOW", fechar_janela)

# Inicia o loop da interface gráfica
janela.mainloop()